# El vector de Galerkin

In [1]:
from sympy import diff, Function, simplify, symbols, factor, Eq, init_printing, trigsimp, expand, latex, Matrix

# Para imprimir bonito
init_printing()
from IPython.display import Math                 
def imprimir (texto1, variable, texto2=""):
    return Math(texto1 +  rf'{latex(variable)}' + texto2)

In [2]:
# constantes y variables
E, nu   = symbols("E, nu")
x, y, z = symbols("x, y, z")
X, Y, Z = symbols("X, Y, Z")

# constantes de Lamé
lamda = (nu*E)/((1 + nu)*(1 - 2*nu))
G     = E/(2*(1 + nu))

# vector de fuerzas másicas
b = Matrix([[ X ],     
            [ Y ],      
            [ Z ]])

Definimos el laplaciano de un campo escalar y vectorial:

In [3]:
# laplaciano de un campo escalar
lapl     = lambda f : diff(f,x,2) + diff(f,y,2) + diff(f,z,2)

# laplaciano de un campo vectorial rectangular (laplaciano vectorial) 
lapl_vec = lambda v : Matrix([[ lapl(v[0]) ],
                              [ lapl(v[1]) ],
                              [ lapl(v[2]) ]])

In [4]:
# gradiente de un campo escalar
grad = lambda f : Matrix([[ diff(f,x) ],
                          [ diff(f,y) ],
                          [ diff(f,z) ]])

In [5]:
# divergencia de un campo vectorial
div = lambda v : diff(v[0],x) + diff(v[1],y) + diff(v[2],z)

El campo vectorial de desplazamientos $\boldsymbol{u}$ se puede definir en términos del vector de Galerkin $\boldsymbol{p}$ como:
\begin{equation*}
\boldsymbol{u} = \frac{2(1 - \nu)\nabla^2 \boldsymbol{p} - \nabla(\nabla \cdot \boldsymbol{p})}{2G},
\end{equation*}
es decir,
\begin{align*}
u(x,y,z) &= \frac{1-\nu}{G} \nabla^2 p_x - \frac{1}{2G} \frac{\partial}{\partial x}(\nabla \cdot \boldsymbol{p}) \\
v(x,y,z) &= \frac{1-\nu}{G} \nabla^2 p_y - \frac{1}{2G} \frac{\partial}{\partial y}(\nabla \cdot \boldsymbol{p}) \\
w(x,y,z) &= \frac{1-\nu}{G} \nabla^2 p_z - \frac{1}{2G} \frac{\partial}{\partial z}(\nabla \cdot \boldsymbol{p})
\end{align*}   

In [6]:
# vector de Galerkin
px = Function("p_x")(x, y, z)
py = Function("p_y")(x, y, z)
pz = Function("p_z")(x, y, z)
p = Matrix([[ px ],
            [ py ],
            [ pz ]])

# definición del campo vectorial de desplazamientos a partir del vector de Galerkin
vec_u = (2*(1 - nu)*lapl_vec(p) - grad(div(p)))/(2*G)
u, v, w = vec_u

Al reemplazar dicha definición del campo vectorial de desplazamientos $\boldsymbol{u}$ en las ecuaciones de Cauchy-Navier

In [7]:
# ecuación de Cauchy-Navier en términos de desplazamientos
eq1 = simplify((lamda + G)*grad(div(vec_u)) + G*lapl_vec(vec_u) + b)

Se obtiene que:
\begin{equation*}
(1 - \nu)\nabla^4 \boldsymbol{p} + \boldsymbol{b} = \boldsymbol{0}
\end{equation*}

In [8]:
# ecuación de Cauchy-Navier en términos del vector de Galerkin
eq2 = (1 - nu)*lapl_vec(lapl_vec(p)) + b

In [9]:
simplify(eq1 - eq2)

De aquí se deduce que si las fuerzas másicas $\boldsymbol{b}$ son nulas, las componentes del vector de Galerkin deben ser funciones biarmónicas:
\begin{equation*}
\nabla^4 \boldsymbol{p} = \boldsymbol{0}
\end{equation*}

A partir de los desplazamientos se obtienen las deformaciones:
\begin{align*}
\varepsilon_x &= \frac{\partial u}{\partial x}   &   \gamma_{xy} &= \frac{\partial u}{\partial y} + \frac{\partial v}{\partial x} \\
\varepsilon_y &= \frac{\partial v}{\partial y}   &   \gamma_{xz} &= \frac{\partial u}{\partial z} + \frac{\partial w}{\partial x} \\
\varepsilon_z &= \frac{\partial w}{\partial z}   &   \gamma_{yz} &= \frac{\partial v}{\partial z} + \frac{\partial w}{\partial y}
\end{align*}

In [10]:
ex  = diff(u, x)
ey  = diff(v, y)
ez  = diff(w, z)
gxy = diff(u, y) + diff(v, x)
gxz = diff(u, z) + diff(w, x)
gyz = diff(v, z) + diff(w, y)

In [11]:
ex2  = ((1 - nu)/G)*lapl(diff(px,x)) - (1/(2*G))*diff(div(p), x, 2)
ey2  = ((1 - nu)/G)*lapl(diff(py,y)) - (1/(2*G))*diff(div(p), y, 2)
ez2  = ((1 - nu)/G)*lapl(diff(pz,z)) - (1/(2*G))*diff(div(p), z, 2)
gxy2 = ((1 - nu)/G)*lapl(diff(px,y) + diff(py,x)) - (1/G)*diff(div(p), x, y)
gxz2 = ((1 - nu)/G)*lapl(diff(px,z) + diff(pz,x)) - (1/G)*diff(div(p), x, z)
gyz2 = ((1 - nu)/G)*lapl(diff(py,z) + diff(pz,y)) - (1/G)*diff(div(p), y, z)

In [12]:
[ simplify(ex  - ex2),
  simplify(ey  - ey2),
  simplify(ez  - ez2),
  simplify(gxy - gxy2),
  simplify(gxz - gxz2),
  simplify(gyz - gyz2) ]

Aplicando las ecuaciones de Lamé obtenemos los esfuerzos:
\begin{align*}
   \sigma_x &= \lambda e + 2 G \varepsilon_x   &   \tau_{xy} = G\gamma_{xy} \\
   \sigma_y &= \lambda e + 2 G \varepsilon_y   &   \tau_{xz} = G\gamma_{xz} \\
   \sigma_z &= \lambda e + 2 G \varepsilon_z   &   \tau_{yz} = G\gamma_{yz}   
\end{align*}

In [13]:
e = ex + ey + ez

In [14]:
sx  = lamda*e + 2*G*ex
sy  = lamda*e + 2*G*ey
sz  = lamda*e + 2*G*ez
txy = G*gxy
txz = G*gxz
tyz = G*gyz

In [15]:
sx2  = 2*(1-nu)*lapl(diff(px,x)) + nu*lapl(div(p)) - diff(div(p), x, 2)
sy2  = 2*(1-nu)*lapl(diff(py,y)) + nu*lapl(div(p)) - diff(div(p), y, 2)
sz2  = 2*(1-nu)*lapl(diff(pz,z)) + nu*lapl(div(p)) - diff(div(p), z, 2)
txy2 = (1 - nu)*lapl(diff(px,y) + diff(py,x)) - diff(div(p), x, y)
txz2 = (1 - nu)*lapl(diff(px,z) + diff(pz,x)) - diff(div(p), x, z)
tyz2 = (1 - nu)*lapl(diff(py,z) + diff(pz,y)) - diff(div(p), y, z)

In [16]:
[ simplify(sx  - sx2),
  simplify(sy  - sy2),
  simplify(sz  - sz2),
  simplify(txy - txy2),
  simplify(txz - txz2),
  simplify(tyz - tyz2) ]